# Ai Mercado
 yo y los demas

##  Descripcion de variables

## Importacion de librerias y bases

### Autenticador del Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Librerias

In [2]:
!pip install pandas-profiling
!pip install pyxlsb
!pip install sweetviz

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.1 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=0d0a0f10bc1bbc70336119523a8d118cd44ab9d01d9edf4aa179f701962593f7
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 44.3 MB/s eta 0:00:00


In [3]:
import os
import re
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import sweetviz as sv
from google.colab import files

### Funciones

In [4]:
def missing_values(data):
  df = pd.DataFrame()
  for col in list(data):
    unique_values = data[col].unique()
    try:
      unique_values = np.sort(unique_values)
    except:
      pass
    nans = round(pd.isna(data[col]).sum()/data.shape[0]*100, 1)
    # zeros = round((data[col] == 0).sum()/data.shape[0]*100, 1)
    df = pd.concat([df, pd.DataFrame([[col, len(unique_values), nans]], columns=['variable', 'unique value', 'NaN %'])], ignore_index=True)
  return df.rename(columns={'NaN %':'Nan %', 'Zeros %': 'Zeros %'}).sort_values('Nan %', ascending=False)


### Ruta de los datos

Ruta de las bases de datos desde el 2013 hasta septiembre de 2024

In [10]:
pd1 = '/content/drive/MyDrive/IA/Proyecto IA/Base de datos final/datos_manizales_completados.csv'


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Cargar bases de datos

Carga de cada una de las bases de datos con eliminacion de columnas vacias

In [12]:
# Datraframe por años
df1 = pd.read_csv(pd1, sep=",", encoding='latin1')


In [13]:
# Eliminacion de Columnas Vacias (Unnamed)
df = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]


In [14]:
df

,Fecha,Grupo,Producto,Mercado,Precio_kg
0,ene-13,CARNES,"Carne de cerdo, espinazo","Manizales, Centro GalerÃ­as",7000.000000
1,ene-13,CARNES,"Carne de cerdo, lomo sin hueso","Manizales, Centro GalerÃ­as",1300.000000
2,ene-13,CARNES,"Carne de cerdo, pernil sin hueso","Manizales, Centro GalerÃ­as",1100.000000
3,ene-13,CARNES,"Carne de cerdo, tocino papada","Manizales, Centro GalerÃ­as",8000.000000
4,ene-13,FRUTAS,Banano criollo,"Manizales, Centro GalerÃ­as",495.000000
...,...,...,...,...,...
10951,dic-23,PROCESADOS,Sal yodada,"Manizales, Centro GalerÃ­as",2040.000000
10952,dic-23,PROCESADOS,Sardinas en lata,"Manizales, Centro GalerÃ­as",1650.300000
10953,ene-23,PROCESADOS,Manteca,"Manizales, Centro GalerÃ­as",1353.027273
10954,ene-23,PROCESADOS,Margarina,"Manizales, Centro GalerÃ­as",1575.709091


In [15]:
from collections import defaultdict

# Crear un defaultdict que inicialice automáticamente listas vacías
productos_precios_dict = defaultdict(list)

# Recorrer las filas del DataFrame
for _, row in df.iterrows():
    producto = row['Producto']
    precio = row['Precio_kg']

    # Añadir directamente el precio al producto (la clave) en el defaultdict
    productos_precios_dict[producto].append(precio)

# Opcional: Imprimir las primeras claves y valores para verificar
for producto, precios in list(productos_precios_dict.items())[:5]:
    print(f"Producto: {producto}, Precios: {precios}")



Producto: Carne de cerdo, espinazo, Precios: [7000.0, 7000.0, 7000.0, 7000.0, 7000.0, 6833.0, 6667.0, 6917.0, 7000.0, 7000.0, 7000.0, 7067.0, 7167.0, 7000.0, 7267.0, 7333.0, 8000.0, 7750.0, 7667.0, 7333.0, 6000.0, 6000.0, 6000.0, 6000.0, 6042.0, 6167.0, 6167.0, 6167.0, 6083.0, 6000.0, 6000.0, 6417.0, 6667.0, 6533.0, 6500.0, 6900.0, 7000.0, 7000.0, 7400.0, 7333.0, 7333.0, 7067.0, 6667.0, 6667.0, 7000.0, 7000.0, 7000.0, 7300.0, 7333.0, 7333.0, 7333.0, 7417.0, 7667.0, 7667.0, 7167.0, 7000.0, 7000.0, 7000.0, 7000.0, 7750.0, 8000.0, 8000.0, 6933.0, 6667.0, 6533.0, 6500.0, 6542.0, 6667.0, 6667.0, 7417.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 7667.0, 8333.0, 8333.0, 8333.0, 8132.999999999999, 8000.0, 7500.0, 7333.0, 7583.0, 8167.0, 8667.0, 9250.0, 9667.0, 9667.0, 9667.0, 9533.0, 9000.0, 9333.0, 9583.0, 1033.3, 1033.3, 1046.7, 1066.7, 9858.3, 9858.3, 1066.7, 1066.7, 1066.7, 1066.7, 1066.7, 1106.7, 1158.3, 1216.7, 1233.3, 1233.3,

In [16]:
# Crear un diccionario vacío para almacenar los productos y sus precios
productos_precios_dict = {}

# Recorrer las filas del DataFrame
for _, row in df.iterrows():
    producto = row['Producto']
    precio = row['Precio_kg']

    # Si el producto ya está en el diccionario, añadimos el precio a su lista
    if producto in productos_precios_dict:
        productos_precios_dict[producto].append(precio)
    # Si no está, creamos una nueva entrada con el producto como clave y el precio en una lista
    else:
        productos_precios_dict[producto] = [precio]

# Mostrar los primeros elementos del diccionario para verificar el resultado
# Seleccionamos solo las primeras 5 claves para evitar mostrar demasiados datos
dict(list(productos_precios_dict.items())[:1])


{'Carne de cerdo, espinazo': [7000.0,
  7000.0,
  7000.0,
  7000.0,
  7000.0,
  6833.0,
  6667.0,
  6917.0,
  7000.0,
  7000.0,
  7000.0,
  7067.0,
  7167.0,
  7000.0,
  7267.0,
  7333.0,
  8000.0,
  7750.0,
  7667.0,
  7333.0,
  6000.0,
  6000.0,
  6000.0,
  6000.0,
  6042.0,
  6167.0,
  6167.0,
  6167.0,
  6083.0,
  6000.0,
  6000.0,
  6417.0,
  6667.0,
  6533.0,
  6500.0,
  6900.0,
  7000.0,
  7000.0,
  7400.0,
  7333.0,
  7333.0,
  7067.0,
  6667.0,
  6667.0,
  7000.0,
  7000.0,
  7000.0,
  7300.0,
  7333.0,
  7333.0,
  7333.0,
  7417.0,
  7667.0,
  7667.0,
  7167.0,
  7000.0,
  7000.0,
  7000.0,
  7000.0,
  7750.0,
  8000.0,
  8000.0,
  6933.0,
  6667.0,
  6533.0,
  6500.0,
  6542.0,
  6667.0,
  6667.0,
  7417.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  7667.0,
  8333.0,
  8333.0,
  8333.0,
  8132.999999999999,
  8000.0,
  7500.0,
  7333.0,
  7583.0,
  8167.0,
  8667.0,
  9250.0,
  9667.0,
 

In [17]:
# Obtener la lista de llaves (productos) del diccionario
lista_llaves = list(productos_precios_dict.keys())

# Contar el número de llaves
cantidad_llaves = len(lista_llaves)

# Imprimir la lista de llaves y la cantidad
print("Lista de llaves:", lista_llaves)
print("Cantidad de llaves:", cantidad_llaves)


Lista de llaves: ['Carne de cerdo, espinazo', 'Carne de cerdo, lomo sin hueso', 'Carne de cerdo, pernil sin hueso', 'Carne de cerdo, tocino papada', 'Banano criollo', 'Coco', 'Fresa', 'Granadilla', 'GuanÃ¡bana', 'Guayaba agria', 'Guayaba manzana', 'Guayaba pera', 'Kiwi', 'LimÃ³n TahitÃ\xad', 'Lulo', 'Mandarina Oneco', 'Mango Tommy', 'Manzana roja importada', 'Manzana royal gala importada', 'Manzana verde importada', 'MaracuyÃ¡', 'MelÃ³n Cantalup', 'Mora de Castilla', 'Naranja Valencia', 'Papaya Maradol', 'Pera importada', 'PiÃ±a gold', 'Tangelo', 'Tomate de Ã¡rbol', 'Uva red globe nacional', 'Arroz de primera', 'Arveja verde seca importada', 'FrÃ\xadjol cargamanto rojo', 'Lenteja importada', 'MaÃ\xadz amarillo cÃ¡scara', 'MaÃ\xadz blanco trillado', 'Huevo blanco A', 'Huevo blanco AA', 'Huevo blanco extra', 'Queso campesino', 'Queso costeÃ±o', 'Bocachico importado congelado', 'AzÃºcar sulfitada', 'CafÃ© molido', 'Chocolate amargo', 'Gelatina', 'Harina de trigo', 'Harina precocida de maÃ

In [18]:
# Contar el número de elementos en cada lista de precios para cada producto
for producto, precios in productos_precios_dict.items():
    # Contar elementos en el vector de precios
    cantidad_precios = len(precios)
    # Imprimir el nombre del producto y la cantidad de precios asociados
    print(f"Producto: {producto}, Cantidad de precios: {cantidad_precios}")


Producto: Carne de cerdo, espinazo, Cantidad de precios: 132
Producto: Carne de cerdo, lomo sin hueso, Cantidad de precios: 132
Producto: Carne de cerdo, pernil sin hueso, Cantidad de precios: 132
Producto: Carne de cerdo, tocino papada, Cantidad de precios: 132
Producto: Banano criollo, Cantidad de precios: 132
Producto: Coco, Cantidad de precios: 132
Producto: Fresa, Cantidad de precios: 132
Producto: Granadilla, Cantidad de precios: 132
Producto: GuanÃ¡bana, Cantidad de precios: 132
Producto: Guayaba agria, Cantidad de precios: 132
Producto: Guayaba manzana, Cantidad de precios: 132
Producto: Guayaba pera, Cantidad de precios: 132
Producto: Kiwi, Cantidad de precios: 132
Producto: LimÃ³n TahitÃ­, Cantidad de precios: 132
Producto: Lulo, Cantidad de precios: 132
Producto: Mandarina Oneco, Cantidad de precios: 132
Producto: Mango Tommy, Cantidad de precios: 132
Producto: Manzana roja importada, Cantidad de precios: 132
Producto: Manzana royal gala importada, Cantidad de precios: 132
P